In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
dir_path = '/content/gdrive/My Drive/Colab_Notebooks/APS360/Project'
player_path =  dir_path + '/nba_summary_17_19.csv'
team_path = dir_path + '/nba_team_boxscores_2017-2019.csv'

In [0]:
import pandas as pd
player_df = pd.read_csv(player_path)
team_df = pd.read_csv(team_path)
#print(player_df.columns, '\n\n',team_df.columns)

In [5]:
# dataframe property
print(player_df.shape, team_df.shape)

(69863, 81) (5538, 63)


In [6]:
# filter match ids
match_id_field = list(player_df['match_id']) + list(team_df['match_id'])
unwanted_ids = [i for i in match_id_field if str(i)[0] != '2']
print("unwanted match_id number: ", len(unwanted_ids))

unwanted match_id number:  9665


In [7]:
# assumption testing -> match id is always 8 digits
match_id_len = [len(str(i)) == 8 for i in match_id_field]
print("Assumption is tested to be: ", all(match_id_len))

Assumption is tested to be:  True


In [0]:
# regular season match ids start with 2
# Filter all those starting with 2
def remove_non_nba(data):
  return data[(data['match_id'] >= 20000000) & (data['match_id'] < 30000000)]

In [10]:
# test match_id filter
nba_player_df = remove_non_nba(player_df)  # nba df is all match within nba(match id starts with 2)
nba_team_df = remove_non_nba(team_df)
print("Rows filtered out: ", (player_df.shape[0] + team_df.shape[0]) - (nba_player_df.shape[0] + nba_team_df.shape[0]))


Rows filtered out:  9665


In [11]:
# Now we try to understand the season data
season_id_field = list(set(player_df['season'])) + list(set(team_df['season_id']))
print("Season ids are: ", season_id_field)

Season ids are:  ['2017-18', '2018-19', 22017, 22018]


In [0]:
# convert season_id for consistency
pd.options.mode.chained_assignment = None

nba_player_df.loc[nba_player_df['season'] == '2018-19', 'season_id'] = 22018
nba_player_df.loc[nba_player_df['season'] == '2017-18', 'season_id'] = 22017
nba_player_df = nba_player_df.astype({'season_id':int})

In [0]:
# The data contains 2 seasons
# Separate 2017 and 2018 season
# Look at season_id attribute
def separate_season(data):
  return data[data['season_id'] == 22017], data[data['season_id'] == 22018]


In [14]:
# separate data based on season
player17_df, player18_df = separate_season(nba_player_df)
team17_df, team18_df = separate_season(nba_team_df)
print(player17_df.shape, player18_df.shape, team17_df.shape, team18_df.shape)

(30020, 81) (30796, 81) (2460, 63) (2460, 63)


In [0]:
# add a home/ away column to team df
def bool_home_away(player, team):
  # Use the players dataset
  # Compare team_id with hometeam_id
  # Then use match_ids in the two datasets to take home/away boolean
  # from players dataset to team dataset
  isHome = player['team_id'] == player['hometeam_id']
  player = player.assign(isHome = isHome.values)
  team = team.merge(player[['match_id', 'team_id', 'isHome']], on=['match_id', 'team_id'], how = 'left')
  team.drop_duplicates(inplace=True)
  return player, team

In [16]:
# add isHome to both player and team df
player17, team17 = bool_home_away(player17_df, team17_df)
player18, team18 = bool_home_away(player18_df, team18_df)
print(player17.shape, player18.shape, team17.shape, team18.shape)

(30020, 82) (30796, 82) (2460, 64) (2460, 64)


In [0]:
def pts_scored_recieved(player):
  player['pts_scored'] = 0 
  player['pts_received'] = 0
  player.loc[player['isHome'] == True, ['pts_scored']] = player['final_hscore'] # points scored = home score for home teams.
  player.loc[player['isHome'] == True, ['pts_received']] = player['final_vscore'] # points received = away score for home teams.
  player.loc[player['isHome'] == False, ['pts_scored']] = player['final_vscore']
  player.loc[player['isHome'] == False, ['pts_received']] = player['final_hscore']
  return player

In [18]:
pts_scored_recieved(player17)
pts_scored_recieved(player18)
print('Done')

Done


In [25]:
team17[team17["match_id"]==21700056][['match_id', 'team_id', 'start_datetime']]

ERROR! Session/line number was not unique in database. History logging moved to new session 59


,match_id,team_id,start_datetime
6182,21700056,1610612750,2017-10-25 23:00:00+00:00
6194,21700056,1610612765,2017-10-25 23:00:00+00:00


In [36]:
dataset = player17.copy()
dataset.shape
dataset = dataset.merge(team17[['match_id', 'team_id', 'start_datetime']], on=['match_id', 'team_id'], how = 'left')
dataset.drop_duplicates(inplace=True)
dataset = dataset.reset_index(drop=True)
dataset[(dataset["team_id"]==1610612750)][['match_id', 'team_id', 'start_datetime']].drop_duplicates(subset=['start_datetime'], keep='first', inplace=False)

,match_id,team_id,start_datetime
281,21700729,1610612750,2018-01-28 02:00:00+00:00
319,21700309,1610612750,2017-11-30 01:00:00+00:00
893,21700238,1610612750,2017-11-20 00:00:00+00:00
1015,21700849,1610612750,2018-02-14 01:00:00+00:00
1115,21700838,1610612750,2018-02-12 00:00:00+00:00
...,...,...,...
28927,21701088,1610612750,2018-03-24 22:00:00+00:00
29112,21701106,1610612750,2018-03-27 00:00:00+00:00
29212,21701120,1610612750,2018-03-29 00:00:00+00:00
29277,21701134,1610612750,2018-03-31 00:30:00+00:00


In [0]:
dataset.sort_values(by=['start_datetime'], inplace=True, ascending=True)

In [139]:
dataset['match_id']

0        21700366
1        21700366
2        21700366
3        21700366
4        21700366
           ...   
33998    21701227
33999    21701227
34000    21701227
34001    21701227
34002    21701227
Name: match_id, Length: 30020, dtype: int64

In [0]:
# adds 2 columns to player dataset, 
# ['start_datetime'] is the time the match took place
# ['rest_hours'] is the time between the current match and their last match in hours
def add_rest_time(team_dataset, player_dataset):
  import numpy as np

  #group and sort based on team_id
  team_dataset['start_datetime'] = pd.to_datetime(team_dataset.start_datetime)
  team_dataset = team_dataset.groupby('team_id').apply(pd.DataFrame.sort_values, ['start_datetime'], ascending=True)
  #rest period for each team is in hours
  team_dataset['rest_hours'] = team_dataset.start_datetime.diff().astype('timedelta64[h]')
  #fix the first entry of each team
  mask = team_dataset.team_id != team_dataset.team_id.shift(1)
  team_dataset['rest_hours'][mask] = np.nan

  #revert index
  team_dataset.reset_index(level=0, inplace=True,drop=True)

  player_dataset = player_dataset.merge(team_dataset[['match_id', 'team_id', 'start_datetime','rest_hours']], on=['match_id', 'team_id'], how = 'left')

  return team_dataset.dropna(subset=['rest_hours']), player_dataset.dropna(subset=['rest_hours'])

In [0]:
dataset = player17.copy()
dataset2 = team17.copy()

In [0]:
team_df_with_rest_time,player_df_with_rest_time = add_rest_time(dataset2,dataset)

In [137]:
team_df_with_rest_time.shape

(2430, 65)

In [0]:
#sanity test, check info from both dataframe match
player_df_match_info = player_df_with_rest_time.drop_duplicates(subset=['match_id','team_id']).sort_values(by=['match_id','team_id'], ascending=True)

team_df_match_info =team_df_with_rest_time.drop_duplicates(subset=['match_id','team_id']).sort_values(by=['match_id','team_id'], ascending=True)


In [183]:
df1 = player_df_match_info['rest_hours']
df2 = team_df_match_info['rest_hours']
index = df1.where(df1.values==df2.values).notna() 
index[index!=True]


Series([], Name: rest_hours, dtype: bool)

In [174]:
team_df_match_info.tail()

,match_id,start_datetime,team_id,team_name_x,team_abbr,team_city,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,to,pf,pts,plus_minus,source,team_name_y,e_off_rating,off_rating,e_def_rating,def_rating,e_net_rating,net_rating,ast_pct,ast_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,e_tm_tov_pct,tm_tov_pct,efg_pct,ts_pct,usg_pct,e_usg_pct,e_pace,pace,pie,team_name,pts_off_tov,pts_2nd_chance,pts_fb,pts_paint,opp_pts_off_tov,opp_pts_2nd_chance,opp_pts_fb,opp_pts_paint,blka,pfd,granularity,season_id,per_mode,isHome,rest_hours
19105,21701228,2018-04-12 02:30:00+00:00,1610612747,Lakers,LAL,Los Angeles,240.0,42,88,0.477,17,39,0.436,14,16,0.875,10,39,49,25,8,6,17,23,115,15.0,https://stats.nba.com,Lakers,112.7,112.7,95.9,97.1,16.8,15.7,0.595,1.47,18.2,0.234,0.788,0.525,16.660,16.7,0.574,0.605,1.0,0.200,103.18,102.5,0.571,Lakers,24.0,11.0,18.0,38.0,9.0,10.0,5.0,64.0,1,17,team_game,22017,PerGame,False,24.0
14044,21701229,2018-04-12 02:30:00+00:00,1610612757,Trail Blazers,POR,Portland,240.0,41,89,0.461,9,24,0.375,11,16,0.688,7,39,46,19,9,9,10,22,102,9.0,https://stats.nba.com,Trail Blazers,103.0,104.1,92.5,95.9,10.5,8.2,0.463,1.90,15.2,0.192,0.695,0.459,10.097,10.2,0.511,0.531,1.0,0.200,99.80,97.5,0.566,Trail Blazers,19.0,7.0,7.0,42.0,12.0,20.0,5.0,46.0,7,19,team_game,22017,PerGame,True,49.0
14031,21701229,2018-04-12 02:30:00+00:00,1610612762,Jazz,UTA,Utah,240.0,33,89,0.371,8,23,0.348,19,24,0.792,14,39,53,18,5,7,14,19,93,-9.0,https://stats.nba.com,Jazz,92.5,95.9,103.0,104.1,-10.5,-8.2,0.545,1.20,13.6,0.305,0.808,0.541,14.916,15.5,0.416,0.467,1.0,0.197,99.80,97.5,0.434,Jazz,12.0,20.0,5.0,46.0,19.0,7.0,7.0,42.0,9,22,team_game,22017,PerGame,False,25.0
20712,21701230,2018-04-12 02:30:00+00:00,1610612745,Rockets,HOU,Houston,240.0,31,82,0.378,13,46,0.283,8,12,0.667,8,32,40,11,2,6,14,24,83,-13.0,https://stats.nba.com,Rockets,89.0,90.2,102.3,104.3,-13.4,-14.1,0.355,0.79,9.8,0.208,0.773,0.464,15.009,15.2,0.457,0.475,1.0,0.199,93.54,92.0,0.303,Rockets,14.0,12.0,16.0,28.0,14.0,10.0,10.0,46.0,3,14,team_game,22017,PerGame,False,24.0
20699,21701230,2018-04-12 02:30:00+00:00,1610612758,Kings,SAC,Sacramento,240.0,38,80,0.475,7,26,0.269,13,20,0.650,6,42,48,22,6,3,11,14,96,13.0,https://stats.nba.com,Kings,102.3,104.3,89.0,90.2,13.4,14.1,0.579,2.00,18.1,0.227,0.792,0.536,11.727,12.0,0.519,0.541,1.0,0.200,93.54,92.0,0.697,Kings,14.0,10.0,10.0,46.0,14.0,12.0,16.0,28.0,6,24,team_game,22017,PerGame,True,50.0


In [175]:
player_df_match_info.tail()

,id,created_on,updated_on,source,is_duplicated,display_name,match_id,team_id,team_abbr,team_city,player_id,player_name,start_position,comment,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,to,pf,pts,plus_minus,ast_pct,ast_ratio,ast_tov,blka,def_rating,dreb_pct,...,net_rating,off_rating,opp_pts_2nd_chance,opp_pts_fb,opp_pts_off_tov,opp_pts_paint,oreb_pct,pace,pfd,pie,pts_2nd_chance,pts_fb,pts_off_tov,pts_paint,reb_pct,tm_tov_pct,ts_pct,usg_pct,granularity,per_mode,season_id,dd2,fp,gp,td3,hometeam_id,hometeam_abbr,visitorteam_id,visitorteam_abbr,final_hscore,final_vscore,player_position_code,player_height,player_weight,season,isHome,pts_scored,pts_received,start_datetime,rest_hours
12912,78517,2018-12-06 17:57:49,2018-12-06 17:57:49,https://stats.nba.com,NaN,NaN,21701228,1610612747,LAL,Los Angeles,201572,Brook Lopez,C,NaN,16.650000,4,7,0.571,3,5,0.6,3,3,1.0,0,2,2,1,0,1,0,1,14,8,0.083,11.1,0.0,0,111.4,0.133,...,26.8,138.2,0,0,3,18,0.0,99.48,1,0.163,0,0,3,2,0.067,0.0,0.841,0.205,player_game,PerGame,22017,NaN,NaN,NaN,NaN,1610612746,LAC,1610612747,LAL,100,115,C,2.13,121.6,2017-18,False,115,100,2018-04-12 02:30:00+00:00,24.0
3399,67109,2018-12-06 17:29:44,2018-12-06 17:29:44,https://stats.nba.com,NaN,NaN,21701229,1610612757,POR,Portland,1626192,Pat Connaughton,NaN,NaN,15.733333,2,3,0.667,1,2,0.5,0,0,0.0,0,1,1,1,0,0,0,3,5,3,0.091,25.0,0.0,1,86.7,0.053,...,10.0,96.7,10,0,0,10,0.0,91.55,0,0.057,3,0,0,2,0.028,0.0,0.833,0.088,player_game,PerGame,22017,NaN,NaN,NaN,NaN,1610612757,POR,1610612762,UTA,102,93,G-F,1.96,95.3,2017-18,True,102,93,2018-04-12 02:30:00+00:00,49.0
3390,67100,2018-12-06 17:29:44,2018-12-06 17:29:44,https://stats.nba.com,NaN,NaN,21701229,1610612762,UTA,Utah,1627777,Georges Niang,NaN,DNP - Coach's Decision,0.000000,0,0,0.000,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.000,0.0,0.0,0,0.0,0.000,...,0.0,0.0,0,0,0,0,0.0,0.00,0,0.000,0,0,0,0,0.000,0.0,0.000,0.000,player_game,PerGame,22017,NaN,NaN,NaN,NaN,1610612757,POR,1610612762,UTA,102,93,F,1.98,104.8,2017-18,False,93,102,2018-04-12 02:30:00+00:00,25.0
10096,75107,2018-12-06 17:49:42,2018-12-06 17:49:42,https://stats.nba.com,NaN,NaN,21701230,1610612745,HOU,Houston,101108,Chris Paul,NaN,DNP - Coach's Decision,0.000000,0,0,0.000,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.000,0.0,0.0,0,0.0,0.000,...,0.0,0.0,0,0,0,0,0.0,0.00,0,0.000,0,0,0,0,0.000,0.0,0.000,0.000,player_game,PerGame,22017,NaN,NaN,NaN,NaN,1610612758,SAC,1610612745,HOU,96,83,G,1.83,79.4,2017-18,False,83,96,2018-04-12 02:30:00+00:00,24.0
10106,75117,2018-12-06 17:49:42,2018-12-06 17:49:42,https://stats.nba.com,NaN,NaN,21701230,1610612758,SAC,Sacramento,203998,Bruno Caboclo,NaN,DNP - Coach's Decision,0.000000,0,0,0.000,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.000,0.0,0.0,0,0.0,0.000,...,0.0,0.0,0,0,0,0,0.0,0.00,0,0.000,0,0,0,0,0.000,0.0,0.000,0.000,player_game,PerGame,22017,NaN,NaN,NaN,NaN,1610612758,SAC,1610612745,HOU,96,83,F,2.06,93.0,2017-18,True,96,83,2018-04-12 02:30:00+00:00,50.0
